In [6]:
import glob
import pandas as pd
import json

projects = ["djanjo", "flask", "pytorch", "pandas"]
dfs=[]

for project in projects:
    filenames = glob.glob(f"../output/pytlint/{project}/*.json")
    for filename in filenames:
        with open(filename) as json_data:
            data = json.load(json_data)
            df = pd.json_normalize(data)
            df['project'] = project
            dfs.append(df)




In [7]:
df_py = pd.concat(dfs, ignore_index=True)

df_py.head(5)

,type,module,obj,line,column,endLine,endColumn,path,symbol,message,message-id,project
0,warning,flask.cli,FlaskGroup.list_commands,605,15,605,24,projects/py/flask/src/flask/cli.py,broad-except,Catching too general exception Exception,W0703,flask
1,warning,flask.cli,run_command,898,11,898,20,projects/py/flask/src/flask/cli.py,broad-except,Catching too general exception Exception,W0703,flask
2,warning,test_reqctx,test_teardown_with_previous_exception,39,11,39,20,projects/py/flask/tests/test_reqctx.py,broad-except,Catching too general exception Exception,W0703,flask
3,warning,test_reqctx,test_teardown_with_handled_exception,58,15,58,24,projects/py/flask/tests/test_reqctx.py,broad-except,Catching too general exception Exception,W0703,flask
4,warning,test_appctx,test_app_tearing_down_with_previous_exception,66,11,66,20,projects/py/flask/tests/test_appctx.py,broad-except,Catching too general exception Exception,W0703,flask


In [8]:
df_py.groupby(['project','message-id'])['message-id'].count()

project  message-id
flask    W0703           9
pandas   W0703          24
         W0706           2
         W0707          22
pytorch  E0702           5
         E0704           2
         W0702           4
         W0703         253
         W0706           6
         W0707           9
         W0715          11
Name: message-id, dtype: int64

In [10]:
import psycopg2

conn = psycopg2.connect("host={} user={} dbname=exception_miner password={} port={}".format("127.0.0.1", "exception_miner", "exception_miner", "5432"))
conn.set_session(autocommit=True)
cur = conn.cursor()

cur.execute("""CREATE TABLE IF NOT EXISTS exceptions_pylint (type VARCHAR(255), module VARCHAR(255), obj VARCHAR(255), beginLine INT, beginColumn INT,	endLine INT, endColumn INT, path VARCHAR(255), symbol VARCHAR(255), message VARCHAR(255), message_id VARCHAR(255), project VARCHAR(255))""")

cur.execute("""DELETE FROM exceptions_pylint;""")

for i, row in df_py.iterrows():
    #print(row)
    cur.execute(("""INSERT INTO exceptions_pylint (type, module, obj, beginLine, beginColumn, endLine, endColumn, path, symbol, message, message_id, project)
                            VALUES (%s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s  );"""), list(row))

type                                           warning
module                                       flask.cli
obj                           FlaskGroup.list_commands
line                                               605
column                                              15
endLine                                            605
endColumn                                           24
path                projects/py/flask/src/flask/cli.py
symbol                                    broad-except
message       Catching too general exception Exception
message-id                                       W0703
project                                          flask
Name: 0, dtype: object
type                                           warning
module                                       flask.cli
obj                                        run_command
line                                               898
column                                              11
endLine                                   

Python Parser

In [2]:
import glob
import pandas as pd
import json

projects = ["django", "flask", "pytorch", "pandas"]
dfs=[]

for project in projects:
    #filenames = glob.glob(f"../output/parser/*.csv")
    #print(filename)
    df = pd.read_csv(f"../output/parser/{project}_stats.csv")
    df['project'] = project
    dfs.append(df)


In [3]:
df = pd.concat(dfs, ignore_index=True)
df.head(5)

,file,function,n_try_except,n_try_pass,n_generic_except,n_captures_broad_raise,n_captures_try_except_raise,n_captures_misplaced_bare_raise,project
0,/home/r4ph/desenv/exception-miner/projects/py/...,_path,0,0,0,0,0,0,django
1,/home/r4ph/desenv/exception-miner/projects/py/...,include,1,0,0,0,0,0,django
2,/home/r4ph/desenv/exception-miner/projects/py/...,_reverse_with_prefix,1,0,0,0,0,0,django
3,/home/r4ph/desenv/exception-miner/projects/py/...,reverse,0,0,0,0,0,0,django
4,/home/r4ph/desenv/exception-miner/projects/py/...,resolve_error_handler,0,0,0,0,0,0,django


In [4]:
df.shape

(119038, 9)

In [5]:
#TODO:
#Analisar Duplicados
df[df.duplicated()]

,file,function,n_try_except,n_try_pass,n_generic_except,n_captures_broad_raise,n_captures_try_except_raise,n_captures_misplaced_bare_raise,project
23,/home/r4ph/desenv/exception-miner/projects/py/...,check,0,0,0,0,0,0,django
24,/home/r4ph/desenv/exception-miner/projects/py/...,__repr__,0,0,0,0,0,0,django
25,/home/r4ph/desenv/exception-miner/projects/py/...,__init__,0,0,0,0,0,0,django
28,/home/r4ph/desenv/exception-miner/projects/py/...,check,0,0,0,0,0,0,django
32,/home/r4ph/desenv/exception-miner/projects/py/...,__init__,0,0,0,0,0,0,django
...,...,...,...,...,...,...,...,...,...
119029,/home/r4ph/desenv/exception-miner/projects/py/...,run,0,0,0,0,0,0,pandas
119030,/home/r4ph/desenv/exception-miner/projects/py/...,initialize_options,0,0,0,0,0,0,pandas
119032,/home/r4ph/desenv/exception-miner/projects/py/...,finalize_options,0,0,0,0,0,0,pandas
119033,/home/r4ph/desenv/exception-miner/projects/py/...,initialize_options,0,0,0,0,0,0,pandas


In [18]:
df = df.drop_duplicates().shape

(97554, 6)

In [21]:
df[df['n_generic_except'] > 1].values[0]

array(['/home/r4ph/desenv/exception-miner/projects/py/django/django/db/backends/mysql/creation.py',
       '_clone_test_db', 2, 0, 2, 'django'], dtype=object)

In [22]:
import psycopg2

conn = psycopg2.connect("host={} user={} dbname=exception_miner password={} port={}".format("127.0.0.1", "exception_miner", "exception_miner", "5432"))
conn.set_session(autocommit=True)
cur = conn.cursor()

cur.execute("""CREATE TABLE IF NOT EXISTS exceptions_parser (file VARCHAR(255), function VARCHAR(255), n_try_except INT, n_try_pass INT, n_generic_except INT, project VARCHAR(255))""")

cur.execute("""DELETE FROM exceptions_parser;""")

for i, row in df.iterrows():
    #print(row)
    cur.execute(("""INSERT INTO exceptions_parser (file, function, n_try_except, n_try_pass, n_generic_except, project)
                            VALUES (%s, %s, %s, %s, %s, %s );"""), list(row))